Influence of the number of viewpoints on the result quality
================================================

In [ ]:
import sys
import os
sys.path.append(os.path.realpath(".."))
from preamble import *
from io_ply import read_ply, write_ply
from geometry import massmatrix_voronoi
sns.set(font_scale=1.1)
sns.set_style({'axes.grid' : False})
sns.set_style("white")

basename = os.path.join(OUTPUT_DIR, os.path.basename(os.getcwd()))
recompute = True

In [ ]:
viewpoints = pd.read_csv(os.path.join(basename, "viewpoints.csv"), index_col=0).values[:,0]

In [ ]:
# Load wireframes
wireframes = [[],[]]
losses = [[], []]
viewpoint = 9
res = 100
for num_vp in viewpoints:
    for i, method in enumerate(["reg", "smooth"]):
        filename = os.path.join(basename, f"res_{num_vp:03d}_{method}_wireframe.png")
        if not os.path.exists(filename) or recompute:
            blender_render(os.path.join(basename, f"res_{num_vp:03d}_{method}.ply"), 
                           basename, 
                           25,
                           9,
                           res=res,
                           wireframe=True,
                           ours=(i==1),
                           baseline=(i==0))
        wireframes[i].append(plt.imread(filename))
        losses[i].append(pd.read_csv(os.path.join(basename, f"loss_{num_vp:03d}_{method}.csv"), index_col=0).values)

In [ ]:
try:
    from igl import hausdorff
    hausdorff_distances = np.zeros((2, len(viewpoints)))
    mesh = read_ply(os.path.join(basename, f"ref.ply"))
    v_ref, f_ref = mesh["vertices"].cpu().numpy(), mesh["faces"].cpu().numpy()
    for j in range(len(viewpoints)):
        for i, method in enumerate(["reg", "smooth"]):
            mesh = read_ply(os.path.join(basename, f"res_{viewpoints[j]:03d}_{method}.ply"))
            v, f = mesh["vertices"].cpu().numpy(), mesh["faces"].cpu().numpy()
            hausdorff_distances[i,j] = hausdorff(v_ref, f_ref, v, f) + hausdorff(v, f, v_ref, f_ref)
except ModuleNotFoundError:
    print("WARNING: could not import libigl. The Hausdorff distances will not be computed. Please install libigl if you want to compute them.")
    hausdorff_distances = None

In [ ]:
fontsize = 18

base_size = 3
n_rows = 2
n_cols = len(viewpoints)
h,w,_ = wireframes[0][0].shape
crop_ratio = 0.88
crop_w = int(crop_ratio*h)

fig = plt.figure(1, figsize=(n_cols * base_size* crop_ratio, n_rows * base_size*1.1), constrained_layout=True)
gs = fig.add_gridspec(n_rows,  n_cols, wspace=0.075,hspace=0.0)

for i, method in enumerate(["Regularized", r"Ours ($\lambda=19$)"]):
    for j in range(n_cols):
        ax = fig.add_subplot(gs[i,j])
        ax.tick_params(
            axis='both',
            which='both',
            labelleft=False,
            labelbottom=False)
        
        im = ax.imshow(wireframes[i][j][:,(w-crop_w)//2:(w+crop_w)//2])
        if hausdorff_distances is not None:
            ax.set_xlabel(f"$H$={hausdorff_distances[i,j]:.3e}", fontsize=fontsize)
        if i == 1:
            ax.set_title(f"{viewpoints[j]}", y=-0.3, fontsize=fontsize)
        if j == 0:
            ax.set_ylabel(method, fontsize=fontsize)

        ax.tick_params(
            axis='both',
            which='both',
            labelleft=False,
            labelbottom=False)
        sns.despine(ax=ax, left=True, bottom=True)

arrow = matplotlib.patches.FancyArrowPatch((0,-1), (1,-1),fc = "g", connectionstyle="arc3,rad=0.2", arrowstyle='simple')
fig.patches.append(arrow)

arrow = matplotlib.patches.FancyArrowPatch(
    (0.05,-0.05), (0.95,-0.05), transform=fig.transFigure, fc='black', mutation_scale = 40.)
fig.patches.append(arrow)

fig.suptitle('Number of viewpoints', y=-0.1, fontsize=fontsize)
plt.savefig(os.path.join(basename, "viewpoints.pdf"), format='pdf', dpi=300, bbox_inches='tight', pad_inches=0.005)